# BQL Credit Spreads

This notebook demonstrates how to use Bloomberg Query Language (BQL) to calculate credit spreads for bond curves.

A **credit spread** measures how much extra yield a corporate bond pays compared to a "risk-free" benchmark like Treasuries or SOFR swaps. Bloomberg's CMX screen shows these spreads, but getting the data programmatically requires knowing which benchmark tickers to use.

In [1]:
import pandas as pd

from blp import Blp

## BQL Basics

BQL is Bloomberg's SQL-like query language. It lets you fetch data for multiple securities in a single request.

Basic syntax:
```
get(<fields>) for(<universe>)
```

Or using the `blp.bql()` method with separate `universe` and `fields` parameters.

In [2]:
with Blp(skip_test=True) as blp:
    resp = blp.bql("get(px_last) for(['AAPL US Equity', 'IBM US Equity'])")
    df = resp.as_dataframe()

df

,ID,px_last,DATE,CURRENCY
0,AAPL US Equity,256.109985,2026-01-08T00:00:00Z,USD
1,IBM US Equity,299.799988,2026-01-08T00:00:00Z,USD


## BVAL Curve Members

Bloomberg's BVAL (Bond Valuation) curves are composite yield curves for different credit ratings. For example:

- `BVSC0570 Index` = USD BB-rated composite curve

The `curvemembers()` function returns all points on the curve with their tenors and yields.

In [3]:
INDEX = 'BVSC0570 Index'

with Blp(skip_test=True) as blp:
    resp = blp.bql(
        universe=f"curvemembers('{INDEX}')",
        fields='id().tenor, px_last'
    )
    curve_df = resp.as_dataframe()

curve_df

,ID,id().tenor,px_last,DATE,CURRENCY
0,BVABDE01 BVLI Index,1Y,4.45421,2026-01-08T00:00:00Z,None
1,BVABDE02 BVLI Index,2Y,4.61339,2026-01-08T00:00:00Z,None
2,BVABDE03 BVLI Index,3Y,4.90136,2026-01-08T00:00:00Z,None
3,BVABDE04 BVLI Index,4Y,5.20682,2026-01-08T00:00:00Z,None
4,BVABDE05 BVLI Index,5Y,5.44621,2026-01-08T00:00:00Z,None
5,BVABDE06 BVLI Index,6Y,5.63490,2026-01-08T00:00:00Z,None
6,BVABDE07 BVLI Index,7Y,5.78983,2026-01-08T00:00:00Z,None
7,BVABDE08 BVLI Index,8Y,5.92266,2026-01-08T00:00:00Z,None
8,BVABDE09 BVLI Index,9Y,6.04599,2026-01-08T00:00:00Z,None
9,BVABDE10 BVLI Index,10Y,6.17245,2026-01-08T00:00:00Z,None


## Built-in Spread Functions

BQL has built-in spread calculations:

- **I-spread** (`spread_type='I'`): Spread over the swap curve (SOFR)
- **G-spread** (`spread_type='G'`): Spread over government bonds

In [4]:
with Blp(skip_test=True) as blp:
    resp = blp.bql(
        universe=f"curvemembers('{INDEX}')",
        fields="id().tenor, spread(spread_type='I').value"
    )
    i_spread_df = resp.as_dataframe()

print('I-Spread (over SOFR):')
i_spread_df

I-Spread (over SOFR):


,ID,id().tenor,spread(spread_type='I').value
0,BVABDE01 BVLI Index,1Y,102.084453
1,BVABDE02 BVLI Index,2Y,130.592768
2,BVABDE03 BVLI Index,3Y,157.871052
3,BVABDE04 BVLI Index,4Y,183.479569
4,BVABDE05 BVLI Index,5Y,201.419636
5,BVABDE06 BVLI Index,6Y,213.399932
6,BVABDE07 BVLI Index,7Y,221.842294
7,BVABDE08 BVLI Index,8Y,228.456175
8,BVABDE09 BVLI Index,9Y,234.359494
9,BVABDE10 BVLI Index,10Y,240.809736


In [5]:
with Blp(skip_test=True) as blp:
    resp = blp.bql(
        universe=f"curvemembers('{INDEX}')",
        fields="id().tenor, spread(spread_type='G').value"
    )
    g_spread_df = resp.as_dataframe()

print('G-Spread (over government bonds):')
g_spread_df

G-Spread (over government bonds):


,ID,id().tenor,spread(spread_type='G').value
0,BVABDE01 BVLI Index,1Y,97.766736
1,BVABDE02 BVLI Index,2Y,114.392477
2,BVABDE03 BVLI Index,3Y,137.655874
3,BVABDE04 BVLI Index,4Y,159.647366
4,BVABDE05 BVLI Index,5Y,175.033015
5,BVABDE06 BVLI Index,6Y,183.404286
6,BVABDE07 BVLI Index,7Y,188.400010
7,BVABDE08 BVLI Index,8Y,193.822330
8,BVABDE09 BVLI Index,9Y,198.294572
9,BVABDE10 BVLI Index,10Y,203.079682


## Manual Spread Calculation

To replicate CMX's spread calculation exactly, we need to use specific benchmark tickers.

The spread formula is simple:

$\displaystyle \text{spread (bps)} = (\text{bond yield} - \text{benchmark yield}) \times 100$

The $\times 100$ converts from percentage points to basis points (1% = 100 bps).

**Example:**
- BVAL 2Y yield: 4.6134%
- Treasury 2Y yield (CT2): 3.4747%
- Spread: $(4.6134 - 3.4747) \times 100 = 113.9$ bps

### Benchmark Ticker Mappings

**Treasury tickers:**
- Short tenors use USGG* indices
- 2Y+ use CT* (on-the-run Treasuries)

**SOFR swap tickers:**
- Sub-1Y uses letter suffix: A=3M, B=6M, C=9M
- 1Y+ uses numeric suffix

In [6]:
TENOR_TO_TREASURY = {
    '3M': 'USGG3M Index',
    '6M': 'USGG6M Index',
    '1Y': 'GB1 Govt',
    '2Y': 'CT2 Govt',
    '3Y': 'CT3 Govt',
    '4Y': 'CT4 Govt',
    '5Y': 'CT5 Govt',
    '6Y': 'CT6 Govt',
    '7Y': 'CT7 Govt',
    '8Y': 'CT8 Govt',
    '9Y': 'CT9 Govt',
    '10Y': 'CT10 Govt',
    '12Y': 'CT12 Govt',
    '15Y': 'CT15 Govt',
    '20Y': 'CT20 Govt',
    '25Y': 'CT25 Govt',
    '30Y': 'CT30 Govt',
}

TENOR_TO_SOFR = {
    '3M': 'USOSFRA Curncy',
    '6M': 'USOSFRB Curncy',
    '1Y': 'USOSFR1 Curncy',
    '2Y': 'USOSFR2 Curncy',
    '3Y': 'USOSFR3 Curncy',
    '4Y': 'USOSFR4 Curncy',
    '5Y': 'USOSFR5 Curncy',
    '6Y': 'USOSFR6 Curncy',
    '7Y': 'USOSFR7 Curncy',
    '8Y': 'USOSFR8 Curncy',
    '9Y': 'USOSFR9 Curncy',
    '10Y': 'USOSFR10 Curncy',
    '12Y': 'USOSFR12 Curncy',
    '15Y': 'USOSFR15 Curncy',
    '20Y': 'USOSFR20 Curncy',
    '25Y': 'USOSFR25 Curncy',
    '30Y': 'USOSFR30 Curncy',
}

TENOR_ORDER = ['3M', '6M', '1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '12Y', '15Y', '20Y', '25Y', '30Y']

### Full Spread Calculation

The `get_curve_spreads()` function:
1. Gets curve member yields at each tenor using BQL
2. Fetches Treasury yields (`YLD_YTM_MID`) for matching tenors
3. Fetches SOFR swap rates for matching tenors
4. Calculates spread = curve_yield - benchmark_yield (in basis points)

In [7]:
def get_curve_spreads(index: str, blp: Blp = None) -> pd.DataFrame:
    """Calculate spreads for a BVAL composite index vs Treasury and SOFR.
    """
    def _calculate(blp: Blp) -> pd.DataFrame:
        resp = blp.bql(
            universe=f"curvemembers('{index}')",
            fields='id().tenor, px_last'
        )
        curve_df = resp.as_dataframe()
        curve_df = curve_df[['ID', 'id().tenor', 'px_last']].copy()
        curve_df.columns = ['ID', 'tenor', 'yield']

        tenors_needed = set(curve_df['tenor'].dropna())
        tsy_tickers = [TENOR_TO_TREASURY[t] for t in tenors_needed if t in TENOR_TO_TREASURY]
        sofr_tickers = [TENOR_TO_SOFR[t] for t in tenors_needed if t in TENOR_TO_SOFR]

        tsy_df = pd.DataFrame()
        if tsy_tickers:
            tsy_resp = blp.get_reference_data(tickers=tsy_tickers, flds=['YLD_YTM_MID', 'PX_LAST'])
            tsy_df = tsy_resp.as_dataframe()

        sofr_df = pd.DataFrame()
        if sofr_tickers:
            sofr_resp = blp.get_reference_data(tickers=sofr_tickers, flds=['PX_LAST'])
            sofr_df = sofr_resp.as_dataframe()

        results = []
        for _, row in curve_df.iterrows():
            tenor = row['tenor']
            curve_yield = row['yield']
            if pd.isna(curve_yield):
                continue

            result = {'tenor': tenor, 'yield': round(curve_yield, 4)}

            tsy_ticker = TENOR_TO_TREASURY.get(tenor)
            if tsy_ticker and not tsy_df.empty and tsy_ticker in tsy_df.index:
                tsy_ytm = tsy_df.loc[tsy_ticker, 'YLD_YTM_MID']
                if pd.isna(tsy_ytm):
                    tsy_ytm = tsy_df.loc[tsy_ticker, 'PX_LAST']
                if pd.notna(tsy_ytm):
                    result['tsy_spread'] = round((curve_yield - tsy_ytm) * 100, 1)

            sofr_ticker = TENOR_TO_SOFR.get(tenor)
            if sofr_ticker and not sofr_df.empty and sofr_ticker in sofr_df.index:
                sofr_rate = sofr_df.loc[sofr_ticker, 'PX_LAST']
                if pd.notna(sofr_rate):
                    result['sofr_spread'] = round((curve_yield - sofr_rate) * 100, 1)

            results.append(result)

        result_df = pd.DataFrame(results)
        if not result_df.empty:
            result_df['tenor_sort'] = result_df['tenor'].map(lambda x: TENOR_ORDER.index(x) if x in TENOR_ORDER else 99)
            result_df = result_df.sort_values('tenor_sort').drop(columns=['tenor_sort'])
        return result_df

    if blp is not None:
        return _calculate(blp)

    with Blp(skip_test=True) as blp:
        return _calculate(blp)

In [8]:
df = get_curve_spreads('BVSC0570 Index')
df

,tenor,yield,tsy_spread,sofr_spread
15,3M,4.2650,67.2,59.2
16,6M,4.4026,82.2,72.9
0,1Y,4.4542,97.5,101.1
1,2Y,4.6134,112.9,129.1
2,3Y,4.9014,135.5,155.6
3,4Y,5.2068,156.1,180.8
4,5Y,5.4462,172.0,198.5
5,6Y,5.6349,NaN,210.4
6,7Y,5.7898,185.2,218.7
7,8Y,5.9227,NaN,225.2


## Interpreting the Results

The output shows:
- **tenor**: Maturity point on the curve
- **yield**: BVAL curve yield at that tenor
- **tsy_spread**: Spread over Treasuries (basis points)
- **sofr_spread**: Spread over SOFR swaps (basis points)

For example, if `tsy_spread` at 2Y is 113.9, the BB-rated curve yields 113.9 basis points more than the 2-year Treasury.